In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [13]:

#生成一个批量大小为2，输入维度为20的随机矩阵
X = torch.rand(2, 20) 
X


tensor([[0.5427, 0.6729, 0.9496, 0.7363, 0.5752, 0.2913, 0.7747, 0.0688, 0.1256,
         0.8333, 0.4765, 0.4786, 0.7756, 0.9832, 0.5156, 0.8070, 0.1648, 0.5128,
         0.6840, 0.1847],
        [0.3510, 0.9294, 0.8209, 0.1577, 0.9668, 0.9486, 0.5227, 0.7630, 0.5786,
         0.6841, 0.2896, 0.4872, 0.8569, 0.7433, 0.5067, 0.8630, 0.9584, 0.6365,
         0.2032, 0.7573]])

In [14]:
#建立一个含有两个全连接层的块，激活函数使用relu。
#nn.Sequential类里面定义了一种特殊的Module，任何一个层或者神经网络都可以把它当成Module的一个子类。
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2178,  0.0945, -0.0955,  0.0162,  0.2919, -0.1499, -0.0225,  0.1602,
         -0.1434, -0.1172],
        [-0.2322,  0.0681,  0.0456, -0.0190,  0.3158, -0.1364, -0.0174, -0.0419,
         -0.1609, -0.0883]], grad_fn=<AddmmBackward0>)

自定义块

In [15]:
#自定义一个nn.Module的子类MLP,这样MLP就继承了nn.Module的函数，这里用MLP来实现上面net的效果
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()   #super()用来调用父类的方法，这里调用的是父类的构造函数
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [18]:
net = MLP()
net(X)

tensor([[ 0.0995, -0.1389, -0.1013,  0.2633,  0.0238,  0.1946, -0.1217, -0.0094,
          0.1360, -0.0011],
        [ 0.1020,  0.0161, -0.1735,  0.1869,  0.0341,  0.1357, -0.1106,  0.1470,
          0.1385,  0.1209]], grad_fn=<AddmmBackward0>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):  #*args是收集参数，相当于把若干个参数打包成一个来传入
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0408, -0.1471, -0.2566,  0.0552,  0.1769, -0.0171, -0.1528, -0.0836,
          0.0006,  0.1066],
        [-0.0026, -0.0498, -0.2938, -0.0484,  0.2257,  0.0376, -0.1334,  0.0063,
          0.1930,  0.1946]], grad_fn=<AddmmBackward0>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0002, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0479, grad_fn=<SumBackward0>)